In [1]:
import pandas as pd

In [8]:
train_df = pd.read_csv('../data/train_dataV2.csv')
val_df = pd.read_csv('../data/val_dataV2.csv')
test_df = pd.read_csv('../data/test_dataV2.csv')

In [9]:
train_df.head()

,CategoryCode,ItemCode,DateID,DailySales,DOW,WOY,DOY
0,category_2,117610,2021-11-06,7,5,44,310
1,category_4,836584,2021-11-18,16,3,46,322
2,category_1,370195,2022-01-24,6,0,4,24
3,category_2,172582,2021-10-30,5,5,43,303
4,category_2,1006009,2021-10-30,5,5,43,303


In [64]:
train_udf = train_df.copy().iloc[:, 1:4]
train_udf.head()

,ItemCode,DateID,DailySales
0,117610,2021-11-06,7
1,836584,2021-11-18,16
2,370195,2022-01-24,6
3,172582,2021-10-30,5
4,1006009,2021-10-30,5


In [85]:
start_date = min(train_udf.DateID)
end_date = max(train_udf.DateID)
def df_func(x):
    _x = x.copy().reset_index(drop=True)
    _x.drop(columns=['ItemCode'], inplace=True)
    _x['DateID'] = pd.to_datetime(_x['DateID'])
    _x = _x.set_index('DateID')
    __x = _x.resample('D').sum().fillna(0)
    return __x

In [87]:
res_df = train_udf.groupby('ItemCode').apply(lambda x: df_func(x)).reset_index()
# train_udf.groupby('ItemCode').apply(lambda x: print(x.index)).reset_index()
res_df.head()

,ItemCode,DateID,DailySales
0,3418,2021-10-01,18
1,3418,2021-10-02,3
2,3418,2021-10-03,8
3,3418,2021-10-04,2
4,3418,2021-10-05,4


In [88]:
I2C_map = pd.Series(train_df.CategoryCode.values,index=train_df.ItemCode).to_dict()
res_df['CategoryCode'] = res_df['ItemCode'].map(I2C_map)

In [89]:
res_df.columns = ['ItemCode', 'Date', 'DailySales', 'CategoryCode']
res_df.head()

,ItemCode,Date,DailySales,CategoryCode
0,3418,2021-10-01,18,category_1
1,3418,2021-10-02,3,category_1
2,3418,2021-10-03,8,category_1
3,3418,2021-10-04,2,category_1
4,3418,2021-10-05,4,category_1


In [90]:
res_df.to_csv('../data/train_dataV3.csv')

---

In [91]:
train_items = set(train_df['ItemCode'])
val_items = set(val_df['ItemCode'])
test_items = set(test_df['ItemCode'])

In [92]:
stage_map = {i: 'validation' for i in val_items}
stage_map.update({i: 'test' for i in test_items})
stage_map.update({836584: 'train-only', 1090249: 'train-only'})

In [113]:
df2 = res_df.copy()
df2.head()

,ItemCode,Date,DailySales,CategoryCode
0,3418,2021-10-01,18,category_1
1,3418,2021-10-02,3,category_1
2,3418,2021-10-03,8,category_1
3,3418,2021-10-04,2,category_1
4,3418,2021-10-05,4,category_1


In [114]:
df2['WoY'] = df2["Date"].dt.weekofyear
df2.head()

C:\Users\deela\AppData\Local\Temp/ipykernel_34440/2620650257.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df2['WoY'] = df2["Date"].dt.weekofyear


,ItemCode,Date,DailySales,CategoryCode,WoY
0,3418,2021-10-01,18,category_1,39
1,3418,2021-10-02,3,category_1,39
2,3418,2021-10-03,8,category_1,39
3,3418,2021-10-04,2,category_1,40
4,3418,2021-10-05,4,category_1,40


In [123]:
df2['WoYY'] = df2.apply(lambda x: f"{str(x.Date).split('-')[0]}-{x.WoY}", axis=1)
# df2.apply(lambda x: print(x['Date'], x.WoY), axis=1)
df2.head()

,ItemCode,Date,DailySales,CategoryCode,WoY,WoYY
0,3418,2021-10-01,18,category_1,39,2021-39
1,3418,2021-10-02,3,category_1,39,2021-39
2,3418,2021-10-03,8,category_1,39,2021-39
3,3418,2021-10-04,2,category_1,40,2021-40
4,3418,2021-10-05,4,category_1,40,2021-40


In [124]:
df3 = df2.loc[:, ['ItemCode', 'DailySales', 'WoYY']]
df3.head()

,ItemCode,DailySales,WoYY
0,3418,18,2021-39
1,3418,3,2021-39
2,3418,8,2021-39
3,3418,2,2021-40
4,3418,4,2021-40


In [127]:
def df_func1(x):
    _x = x.copy().reset_index(drop=True)
    _x.drop(columns=['ItemCode'], inplace=True)
    print(_x)
    __x = _x.groupby('WoYY').sum().reset_index()
    print(__x)
    return __x

In [130]:
df5 = df3.groupby(['ItemCode', 'WoYY']).sum().reset_index()
df5.to_csv('../data/train_dataV5.csv')

In [128]:
df4 = df3.groupby('ItemCode').apply(lambda x: df_func1(x))
df4.head()

     DailySales     WoYY
0            18  2021-39
1             3  2021-39
2             8  2021-39
3             2  2021-40
4             4  2021-40
..          ...      ...
131          26   2022-6
132           9   2022-6
133           2   2022-6
134          19   2022-6
135           3   2022-6

[136 rows x 2 columns]
       WoYY  DailySales
0   2021-39          29
1   2021-40          42
2   2021-41          41
3   2021-42          41
4   2021-43          44
5   2021-44          46
6   2021-45          43
7   2021-46          49
8   2021-47          54
9   2021-48          69
10  2021-49         101
11  2021-50          73
12  2021-51          75
13  2021-52          39
14   2022-1          83
15   2022-2          85
16   2022-3          89
17   2022-4          67
18   2022-5         109
19  2022-52          17
20   2022-6          72
     DailySales     WoYY
0             2  2021-39
1             7  2021-39
2             2  2021-39
3             3  2021-40
4             9  2021-4

WoYY  DailySales
ItemCode                       
3418     0  2021-39          29
         1  2021-40          42
         2  2021-41          41
         3  2021-42          41
         4  2021-43          44

In [100]:
df4.to_csv('../data/train_dataV4.csv')